# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки необходимо сравнить поведение пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Импорт необходимых библиотек:

In [1]:
# импорт библиотеки pandas
import pandas as pd

Импорт данных:

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/yandex_music_project.csv')
df.to_csv('yandex_music_project.csv')

In [3]:
# получение первых 10 строк таблицы df
display(df.head(10))

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Информация о данных:

In [4]:
# получение общей информации о данных в таблице df
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB
None
          userID  Track     artist  genre   City        time     Day
count      65079  63848      57876  63881    65079     65079   65079
unique     41748  47245      43605    289        2     20392       3
top     A8AE9169  Intro  Existence    pop   Moscow  08:14:07  Friday
freq          76     34          6   8850    45360        14   23149


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Если в названии колонки больше одного слова, предпочтительнее их разделять нижним подчеркиванием ( _ ).



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Стиль заголовков

In [5]:
# перечень названий столбцов таблицы df
display(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# переименование столбцов
df = df.rename(columns={'  userID' : 'user_id', 
                        'Track' : 'track', 
                        '  City  ' : 'city', 
                        'Day' : 'day'})

In [7]:
# проверка результатов - перечень названий столбцов
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


### Пропуски значений

In [8]:
# подсчёт пропусков
print(df.isna().sum())

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64


Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для данного исследования. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
df[columns_to_replace] = df[columns_to_replace].fillna('unknown')

In [10]:
# подсчёт пропусков
print(df.isna().sum())

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64


### Дубликаты

In [11]:
# подсчёт явных дубликатов
print(f'Количество явных дубликатов: {df.duplicated().sum()}')

Количество явных дубликатов: 3826


In [12]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
# проверка на отсутствие дубликатов
print(f'Количество явных дубликатов: {df.duplicated().sum()}')

Количество явных дубликатов: 0


Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

In [14]:
# Просмотр уникальных названий жанров
display(df['genre'].sort_values().unique())

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

In [15]:
# Функция для замены неявных дубликатов
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

In [16]:
# Устранение неявных дубликатов
replace_wrong_genres(['hip-hop', 'hip', 'hop'], 'hiphop')
replace_wrong_genres(['электроника'], 'electronic')

In [17]:
# Проверка на неявные дубликаты
display(df['genre'].sort_values().unique())

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Заголовки исправлены, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменены на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [18]:
# Подсчёт прослушиваний в каждом городе
print(df.groupby('city')['track'].count())

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Однако, для подтверждения истинности данного предположения, необходимо уточнить количество пользователей, зарегистрированных в Москве и Санкт-Петербурге

In [19]:
# Подсчёт прослушиваний в каждый из трёх дней
print(df.groupby('day')['track'].count())

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [20]:
# <создание функции number_tracks()>
# Объявляется функция с двумя параметрами: day, city.
# В переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city (используйте последовательную фильтрацию
# с помощью логической индексации).
# В переменной track_list_count сохраняется число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.
# Функция возвращает число - значение track_list_count.

def number_tracks(day, city):
    '''Функция для подсчёта прослушиваний для конкретного города и дня. 
    С помощью последовательной фильтрации с логической индексацией она 
    сначала получит из исходной таблицы строки с нужным днём,
    затем из результата отфильтрует строки с нужным городом,
    методом count() посчитает количество значений в колонке user_id. 
    Это количество функция вернёт в качестве результата'''
    
    track_list = df[df['city'] == city]
    track_list = track_list[track_list['day'] == day]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [21]:
number_tracks_moscow = ['Moscow']
number_tracks_spb = ['Saint-Petersburg']

In [22]:
# количество прослушиваний в Москве по понедельникам
number_tracks_moscow.append(number_tracks('Monday', 'Moscow'))
print(f'Количество прослушиваний в Москве по понедельникам: {number_tracks_moscow[1]}')

Количество прослушиваний в Москве по понедельникам: 15740


In [23]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks_spb.append(number_tracks('Monday', 'Saint-Petersburg'))
print(f'Количество прослушиваний в Санкт-Петербурге по понедельникам: {number_tracks_spb[1]}' )

Количество прослушиваний в Санкт-Петербурге по понедельникам: 5614


In [24]:
# количество прослушиваний в Москве по средам
number_tracks_moscow.append(number_tracks('Wednesday', 'Moscow'))
print(f'Количество прослушиваний в Москве по средам: {number_tracks_moscow[2]}')

Количество прослушиваний в Москве по средам: 11056


In [25]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks_spb.append(number_tracks('Wednesday', 'Saint-Petersburg'))
print(f'Количество прослушиваний в Санкт-Петербурге по средам: {number_tracks_spb[2]}' )

Количество прослушиваний в Санкт-Петербурге по средам: 7003


In [26]:
# количество прослушиваний в Москве по пятницам
number_tracks_moscow.append(number_tracks('Friday', 'Moscow'))
print(f'Количество прослушиваний в Москве по пятницам: {number_tracks_moscow[3]}')

Количество прослушиваний в Москве по пятницам: 15945


In [27]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks_spb.append(number_tracks('Friday', 'Saint-Petersburg'))
print(f'Количество прослушиваний в Санкт-Петербурге по пятницам: {number_tracks_spb[3]}' )

Количество прослушиваний в Санкт-Петербурге по пятницам: 5895


In [28]:
for weekday, city in zip(['Monday', 'Wednesday', 'Friday']*2, sorted(['Moscow', 'Saint-Petersburg']*3)):
    print(f'Количество прослушиваний в {city} по {weekday} равно {number_tracks(weekday, city)}')

Количество прослушиваний в Moscow по Monday равно 15740
Количество прослушиваний в Moscow по Wednesday равно 11056
Количество прослушиваний в Moscow по Friday равно 15945
Количество прослушиваний в Saint-Petersburg по Monday равно 5614
Количество прослушиваний в Saint-Petersburg по Wednesday равно 7003
Количество прослушиваний в Saint-Petersburg по Friday равно 5895


In [29]:
# Таблица с результатами
first_hyp_table_columns = ['city', 'monday', 'wednesday', 'friday']
first_hyp_table_data = [number_tracks_moscow, number_tracks_spb]
first_hyp_table = pd.DataFrame(data=first_hyp_table_data, columns=first_hyp_table_columns)
display(first_hyp_table)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


Дополнительно ознакомимся с данным распределением в относительных величинах:

In [30]:
number_tracks_moscow_percent = ['Moscow']
number_tracks_spb_percent = ['Saint-Petersburg']
for i in range(1,len(number_tracks_moscow)):
    number_tracks_moscow_percent.append(number_tracks_moscow[i] / sum(number_tracks_moscow[1:]) * 100)
    number_tracks_spb_percent.append(number_tracks_spb[i] / sum(number_tracks_spb[1:]) * 100)
first_hyp_table_data_percent = [number_tracks_moscow_percent, number_tracks_spb_percent]
first_hyp_table_percent = pd.DataFrame(data=first_hyp_table_data_percent, columns=first_hyp_table_columns)
display(first_hyp_table_percent)

,city,monday,wednesday,friday
0,Moscow,36.826466,25.867434,37.306100
1,Saint-Petersburg,30.326275,37.829516,31.844209


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы, однако, следует принимать во внимание, что разница между днями недели по обоим городам не превышает 13%

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [31]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [32]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

In [33]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
# 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    Используйте последовательную фильтрацию с помощью логической индексации.
# 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
#    столбцов и посчитать методом count() количество записей для каждого из
#    присутствующих жанров, получившийся Series записать в переменную
#    genre_df_count
# 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
#    в переменную genre_df_sorted
# 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)

def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[table['time'] > time1]
    genre_df = genre_df[table['time'] < time2]
    genre_df_count = genre_df.groupby('genre')['user_id'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    return genre_df_sorted[:10]

Сравним результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [34]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
# объекты, хранящие время, являются строками и сравниваются как строки
# пример вызова: genre_weekday(moscow_general, 'Monday', '07:00', '11:00')
print('Топ-10 прослушиваемых жанров в Москве с 07:00 до 11:00 понедельника:')
display(genre_weekday(moscow_general, 'Monday', '07:00', '11:00'))

Топ-10 прослушиваемых жанров в Москве с 07:00 до 11:00 понедельника:


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

In [35]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
print('Топ-10 прослушиваемых жанров в Санкт-Петербурге с 07:00 до 11:00 понедельника :')
display(genre_weekday(spb_general, 'Monday', '07:00', '11:00'))

Топ-10 прослушиваемых жанров в Санкт-Петербурге с 07:00 до 11:00 понедельника :


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

In [36]:
# вызов функции для вечера пятницы в Москве
print('Топ-10 прослушиваемых жанров в Москве с 17:00 до 23:00 пятницы :')
display(genre_weekday(moscow_general, 'Friday', '17:00', '23:00'))

Топ-10 прослушиваемых жанров в Москве с 17:00 до 23:00 пятницы :


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

In [37]:
# вызов функции для вечера пятницы в Петербурге
print('Топ-10 прослушиваемых жанров в Санкт_Петербурге с 17:00 до 23:00 пятницы :')
display(genre_weekday(spb_general, 'Friday', '17:00', '23:00'))

Топ-10 прослушиваемых жанров в Санкт_Петербурге с 17:00 до 23:00 пятницы :


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: user_id, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [38]:
# одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre')['user_id'].count().sort_values(ascending=False)

In [39]:
# просмотр первых 10 строк moscow_genres
display(moscow_genres.to_frame().head(10))

,user_id
genre,
pop,5892
dance,4435
rock,3965
electronic,3786
hiphop,2096
classical,1616
world,1432
alternative,1379
ruspop,1372


In [40]:
spb_genres = spb_general.groupby('genre')['user_id'].count().sort_values(ascending=False)

In [41]:
# просмотр первых 10 строк spb_genres
display(spb_genres.head(10))

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: user_id, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Проверено три предположения и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге, **add** однако, активность в пределах одного города в зависимости от дня недели изменяется не более чем на 13% 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных.